In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from transformers import RobertaTokenizer, RobertaModel  # 使用 RoBERTa
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  # 用于可视化
from tqdm import tqdm  # 用于显示训练进度条
from torch.optim.lr_scheduler import ReduceLROnPlateau  # 学习率调度器
from torch.utils.tensorboard import SummaryWriter  # 导入 TensorBoard 相关的库

# 配置设备
device = torch.device("cpu") 

# 图像数据加载类
class ImageOnlyDataset(Dataset):
    '''
    加载图像数据并与标签进行对应
    '''
    def __init__(self, data_folder, label_file, transform=None):
        self.data_folder = data_folder
        self.transform = transform
        self.data = pd.read_csv(label_file)
        self.guid_list = self.data["guid"].tolist()  # 获取所有的guid
        self.labels = self.data["tag"].map({"positive": 0, "neutral": 1, "negative": 2}).tolist()  # 标签映射为数字

    def __len__(self):
        return len(self.guid_list)

    def __getitem__(self, idx):
        '''
        获取指定索引的数据样本（图像和标签）
        '''
        guid = self.guid_list[idx]
        image_path = os.path.join(self.data_folder, f"{guid}.jpg")

        # 加载图像
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        # 返回图像数据和标签
        label = self.labels[idx]
        return image, label

# 图像预处理
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.RandomRotation(30),  # 随机旋转
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 随机颜色变化
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 加载数据
data_folder = './data'  
label_file = './train.txt'
train_dataset = ImageOnlyDataset(data_folder, label_file, transform=transform)

# 划分训练集和验证集
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# 定义仅图像模型（ResNet50）
class ImageOnlyModel(nn.Module):
    '''
    该模型只使用 ResNet50 来提取图像特征并进行分类。
    1. 使用预训练的 ResNet50 提取图像特征。
    2. 输出3个类别：positive, neutral, negative。
    '''
    def __init__(self):
        super(ImageOnlyModel, self).__init__()

        # 使用预训练的 ResNet50 提取图像特征
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # 去掉全连接层

        # 融合后的分类层
        self.fc = nn.Sequential(
            nn.Linear(2048, 512),  # ResNet50 输出2048维，映射到512维
            nn.ReLU(),
            nn.Dropout(0.5),  # 防止过拟合
            nn.Linear(512, 3)  # 输出3个类别：positive, neutral, negative
        )

    def forward(self, image):
        '''
        前向传播：图像通过 ResNet50 提取特征，后续通过全连接层进行分类
        '''
        image_features = self.resnet(image)  # 提取图像特征
        output = self.fc(image_features)  # 分类
        return output

# 创建 TensorBoard 记录器
writer = SummaryWriter(log_dir='./runs/image_only_model')  # 指定日志保存路径

# 训练模型
def train_model(model, train_loader, val_loader, epochs=5, learning_rate=1e-5, save_path="image_only_model.pth"):
    '''
    该函数用于训练仅图像模型：
    1. 定义优化器和损失函数（交叉熵损失）。
    2. 每个 epoch 中计算训练损失和准确率。
    3. 评估模型在验证集上的表现。
    4. 保存训练过程中最好的模型。
    '''
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)  # L2 正则化
    criterion = nn.CrossEntropyLoss()
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.1, verbose=True)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct_train = 0
        total_train = 0

        # 使用 tqdm 显示训练进度条
        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}", unit="batch"):
            optimizer.zero_grad()

            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        train_losses.append(total_loss / len(train_loader))
        train_accuracy = correct_train / total_train * 100
        train_accuracies.append(train_accuracy)

        # 验证集评估
        model.eval()
        val_loss = 0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}/{epochs}", unit="batch"):
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)

        val_losses.append(val_loss / len(val_loader))
        val_accuracy = correct_val / total_val * 100
        val_accuracies.append(val_accuracy)

        # 在 TensorBoard 中记录训练和验证损失与准确率
        writer.add_scalar('Training Loss', train_losses[-1], epoch)
        writer.add_scalar('Validation Loss', val_losses[-1], epoch)
        writer.add_scalar('Training Accuracy', train_accuracies[-1], epoch)
        writer.add_scalar('Validation Accuracy', val_accuracies[-1], epoch)

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_losses[-1]}, Train Accuracy: {train_accuracies[-1]:.2f}%")
        print(f"Validation Loss: {val_losses[-1]}, Validation Accuracy: {val_accuracies[-1]:.2f}%")

        torch.save(model.state_dict(), save_path)

        # 更新学习率
        scheduler.step(val_losses[-1])

    # 绘制训练过程的损失和准确率曲线
    plot_training_results(train_losses, val_losses, train_accuracies, val_accuracies)

    # 关闭 TensorBoard 记录器
    writer.close()

# 训练模型之前，实例化模型
model = ImageOnlyModel().to(device)

# 训练模型
train_model(model, train_loader, val_loader, epochs=5, learning_rate=1e-5)